In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

sample_submission.csv
test.csv
train.csv



In [2]:
from sklearn.preprocessing import normalize
def get_X_y(filename):
    df = pd.read_csv("../input/" + filename)
    y = np.log(df.SalePrice + 1)
    df = df.drop(['LotFrontage','MasVnrArea','GarageYrBlt','SalePrice','OverallQual'], axis=1)
    df_num = df.loc[:,df.dtypes!='object']
    df_obj = df.loc[:,df.dtypes=='object']
    df_num = pd.DataFrame(normalize(df_num, axis=0), columns = df_num.columns)
    dummies = pd.get_dummies(df_obj)
    X = pd.concat([df_num, dummies], axis=1)
    print(X.isnull().sum()[X.isnull().sum()!=0])
    print("X shape: ", X.shape, ", y shape: ", y.shape)
    return X, y

def rmsle(preds, y):
    sum = 0.0
    for x in range(len(preds)):
        p = np.log(preds[x] + 1)
        r = np.log(y[x] + 1)
        sum = sum + (p - r)**2
    return (sum/len(preds))**0.5

def rmsle_tf(preds, y):
#     return tf.sqrt(tf.reduce_mean(tf.square( tf.log(tf.add(preds,tf.constant(1.0, dtype=tf.float32))) - tf.log(tf.add(y,tf.constant(1.0, dtype=tf.float32)) ))))
    return tf.sqrt(tf.reduce_mean(tf.square( preds - y)))
#     return tf.sqrt(tf.reduce_mean(tf.square))
# print(df_num.shape, dummies.shape)
# for col in df_obj:
#     print(col, ":", df_obj[col].unique())

In [3]:
from sklearn.model_selection import train_test_split
X, y = get_X_y('train.csv')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.shape

Series([], dtype: int64)
X shape:  (1460, 285) , y shape:  (1460,)


(1168, 285)

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import *
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
from math import sqrt

def regressor(model):
    model.fit(X_train,y_train)
    preds = model.predict(X_test)
#     score = rmsle(preds, y_test.values)
    print("Score: ", model.score(X_test, y_test), ", rmse: ", np.sqrt(mean_squared_error(y_test.values, preds)))

lr = LinearRegression()
rf = RandomForestRegressor(n_estimators=100)
regressor(rf)

Score:  0.857312071361 , rmse:  0.154140393059


In [5]:
regressor(Lasso())

Score:  -8.57802215091e-05 , rmse:  0.40807639273


In [6]:
regressor(Ridge())

Score:  0.838661111825 , rmse:  0.163905065342


In [7]:
import xgboost
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)
regressor(xgb)

Score:  0.85647707085 , rmse:  0.154590744958


In [8]:
from sklearn.svm import SVR
svr = SVR()
regressor(svr)

Score:  0.779933986176 , rmse:  0.19142529887


# TensorFlow

In [ ]:
import tensorflow as tf
y_data = np.array([(y_train)])
y_data = np.transpose(np.array(y_data))

y_test_data = [(y_test)]
y_test_data = np.transpose(np.array(y_test))

X = tf.placeholder(tf.float32, shape=[None, 285])
Y = tf.placeholder(tf.float32, shape=[None, 1])

beta = 0.001
h1 = 200
h2 = 1
h3 = 1
with tf.name_scope('layer1'):
    w1 = tf.Variable(tf.random_normal([285,h1]))
    b1 = tf.Variable(tf.random_normal([h1]))
    l1 = tf.matmul(X,w1) + b1
    l1 = tf.nn.relu(l1)
    w2 = tf.Variable(tf.random_normal([h1,h2]))
    b2 = tf.Variable(tf.random_normal([h2]))
    l2 = tf.matmul(l1,w2) + b2
#     l2 = tf.nn.relu(l2)
#     w3 = tf.Variable(tf.random_normal([h2,h3]))
#     b3 = tf.Variable(tf.random_normal([h3]))
    hypothesis = l2 # tf.matmul(l2,w3) + b3

    
# cost = tf.reduce_mean(tf.square(hypothesis - Y))
# cost = tf.metrics.root_mean_squared_error(hypothesis, Y))
cost = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(Y,hypothesis))))
# regularizer = tf.nn.l2_loss(w1)
# cost = tf.reduce_mean(cost + beta * regularizer)
# tf.metrics.root_mean_squared_error(Y, hypothesis)
optimizer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)

rmse = lambda x, y: np.sqrt(np.mean((x-y)**2))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(7001):
        cost_val, hy_val,_= sess.run(
            [cost, hypothesis, optimizer], feed_dict={X: X_train, Y: y_data})
        if step % 500 == 0:
#             print("train cost: ", tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(y_data,hy_val)))).eval())
            preds = sess.run(hypothesis, feed_dict={X: X_test})            
            print(step)
            print("train cost: ", cost_val)
            print("test cost: ", tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(y_test_data,preds)))).eval())

    print("finished!")

0
train cost:  63.6942
test cost:  325.426298163


500
train cost:  0.245627
test cost:  0.730874551057


1000
train cost:  0.22862
test cost:  0.733986301792


1500
train cost:  0.216578
test cost:  0.740499730488


2000
train cost:  0.203468
test cost:  0.748080768247


...

5000
train cost:  0.126857
test cost:  0.687966377363


5500
train cost:  0.121896
test cost:  0.678628298358


6000
train cost:  0.117855
test cost:  0.670262975933


6500
train cost:  0.115199
test cost:  0.663712139098


7000
train cost:  0.113112
test cost:  0.659373480486
finished!
